In [1]:
import pandas as pd
import numpy as np
import os

# Set pandas display options for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
def load_raw_data(file_path='../data/raw/raw_data.csv'):
    """Load the raw supplier data"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Raw data file not found: {file_path}")
    
    print(f"Loading data from {file_path}")
    df = pd.read_csv(file_path)
    print(f"Loaded {len(df):,} rows with {len(df.columns)} columns")
    return df

df = load_raw_data()
print("\nFirst 5 rows preview:")
df.head()

Loading data from ../data/raw/raw_data.csv
Loaded 2,994 rows with 21 columns

First 5 rows preview:


,date,year,month,quarter,supplier_id,company_name,company_size,industry,location,delivery_reliability,cost_efficiency,defect_rate,quality_score,on_time_delivery_rate,order_volume,response_time_hours,customer_satisfaction,overall_score,performance_category,defect_cost_impact,efficiency_ratio
0,2015-01-01,2015,1,Q1,SUP_001,Fowler Corp,Medium,Machinery,Europe,78.844749,66.314619,1.362205,86.623620,75.458909,316.0,21.961844,7.926791,79.83,Good,21522.84,1.306
1,2015-02-01,2015,2,Q1,SUP_001,Fowler Corp,Medium,Machinery,Europe,75.012850,66.959076,1.026724,84.857776,75.355035,352.0,22.700895,7.965200,79.27,Good,18070.35,1.267
2,2015-03-01,2015,3,Q1,SUP_001,Fowler Corp,Medium,Machinery,Europe,71.885296,66.951447,0.325915,83.383238,74.145862,273.0,22.536271,7.667606,79.22,Good,4448.74,1.245
3,2015-04-01,2015,4,Q2,SUP_001,Fowler Corp,Medium,Machinery,Europe,70.976603,66.916719,1.519901,82.403364,68.687178,330.0,18.246897,7.102901,75.79,Good,25078.36,1.231
4,2015-05-01,2015,5,Q2,SUP_001,Fowler Corp,Medium,Machinery,Europe,69.026941,67.420544,2.166598,82.165795,66.287426,348.0,24.433198,7.535101,74.48,Fair,37698.81,1.219


In [5]:
def check_missing_values(df):
    """Check for missing values in the dataset"""
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df)) * 100
    
    missing_summary = pd.DataFrame({
        'Missing Count': missing_counts,
        'Missing Percentage': missing_percentages
    })
    
    missing_summary = missing_summary[missing_summary['Missing Count'] > 0]
    
    if len(missing_summary) == 0:
        print("No missing values found in the dataset")
        return df
    else:
        print("Missing values found:")
        print(missing_summary)
        return df

# Execute: Check missing values
df = check_missing_values(df)

No missing values found in the dataset


In [6]:
def check_data_types(df):
    """Check and validate data types"""
    print("Current data types:")
    print(df.dtypes)
    
    # Convert date column to datetime if it exists
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        print("Converted 'date' column to datetime")
    
    # Ensure numeric columns are numeric
    numeric_columns = [
        'delivery_reliability', 'cost_efficiency', 'defect_rate',
        'quality_score', 'on_time_delivery_rate', 'order_volume',
        'response_time_hours', 'customer_satisfaction', 'overall_score'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print("Validated numeric columns")
    return df

df = check_data_types(df)
print("\nData types after validation:")
df.dtypes

Current data types:
date                      object
year                       int64
month                      int64
quarter                   object
supplier_id               object
company_name              object
company_size              object
industry                  object
location                  object
delivery_reliability     float64
cost_efficiency          float64
defect_rate              float64
quality_score            float64
on_time_delivery_rate    float64
order_volume             float64
response_time_hours      float64
customer_satisfaction    float64
overall_score            float64
performance_category      object
defect_cost_impact       float64
efficiency_ratio         float64
dtype: object
Converted 'date' column to datetime
Validated numeric columns

Data types after validation:


date                     datetime64[ns]
year                              int64
month                             int64
quarter                          object
supplier_id                      object
company_name                     object
company_size                     object
industry                         object
location                         object
delivery_reliability            float64
cost_efficiency                 float64
defect_rate                     float64
quality_score                   float64
on_time_delivery_rate           float64
order_volume                    float64
response_time_hours             float64
customer_satisfaction           float64
overall_score                   float64
performance_category             object
defect_cost_impact              float64
efficiency_ratio                float64
dtype: object

In [7]:
def check_duplicates(df):
    """Check for and remove duplicate rows"""
    initial_rows = len(df)
    
    # Check for complete duplicates
    duplicate_count = df.duplicated().sum()
    print(f"Found {duplicate_count} complete duplicate rows")
    
    if duplicate_count > 0:
        df = df.drop_duplicates()
        print(f"Removed {duplicate_count} duplicate rows")
    
    # Check for duplicates by supplier and date (should be unique)
    if 'supplier_id' in df.columns and 'date' in df.columns:
        supplier_date_duplicates = df.duplicated(subset=['supplier_id', 'date']).sum()
        if supplier_date_duplicates > 0:
            print(f"Warning: {supplier_date_duplicates} supplier-date duplicates found")
            # Remove keeping the last occurrence (most recent)
            df = df.drop_duplicates(subset=['supplier_id', 'date'], keep='last')
            print(f"Removed supplier-date duplicates, keeping most recent")
    
    final_rows = len(df)
    print(f"Final dataset: {final_rows:,} rows (removed {initial_rows - final_rows:,} duplicates)")
    
    return df

df = check_duplicates(df)

Found 0 complete duplicate rows
Final dataset: 2,994 rows (removed 0 duplicates)


In [8]:
def validate_ranges(df):
    """Validate that numeric values are within expected ranges"""
    validations = {
        'delivery_reliability': (0, 100),
        'cost_efficiency': (0, 100),
        'defect_rate': (0, 10),
        'quality_score': (0, 100),
        'on_time_delivery_rate': (0, 100),
        'order_volume': (0, float('inf')),
        'response_time_hours': (0, 168),  # Max 1 week
        'customer_satisfaction': (0, 10),
        'overall_score': (0, 100)
    }
    
    issues_found = False
    
    for col, (min_val, max_val) in validations.items():
        if col in df.columns:
            out_of_range = ((df[col] < min_val) | (df[col] > max_val)).sum()
            if out_of_range > 0:
                print(f"⚠️  {col}: {out_of_range:,} values outside range [{min_val}, {max_val}]")
                issues_found = True
                
                # Clip values to valid range
                df[col] = df[col].clip(min_val, max_val)
                print(f"Clipped {col} values to valid range")
    
    if not issues_found:
        print("All numeric values within expected ranges")
    
    return df

df = validate_ranges(df)

All numeric values within expected ranges


In [9]:
def clean_text_columns(df):
    """Clean and standardize text columns"""
    text_columns = ['company_name', 'industry', 'location', 'company_size']
    
    for col in text_columns:
        if col in df.columns:
            # Strip whitespace
            df[col] = df[col].str.strip()
            
            # Handle empty strings
            empty_count = (df[col] == '').sum()
            if empty_count > 0:
                print(f"{col}: {empty_count:,} empty strings found")
                # Fill with 'Unknown' or appropriate default
                if col == 'company_name':
                    df[col] = df[col].replace('', 'Unknown Company')
                elif col == 'industry':
                    df[col] = df[col].replace('', 'Unknown')
                elif col == 'location':
                    df[col] = df[col].replace('', 'Unknown')
                elif col == 'company_size':
                    df[col] = df[col].replace('', 'Unknown')
    
    print("Cleaned text columns")
    return df

df = clean_text_columns(df)

Cleaned text columns


In [10]:
def final_validation(df):
    
    # Check final missing values
    final_missing = df.isnull().sum().sum()
    if final_missing == 0:
        print("No missing values remaining")
    else:
        print(f"{final_missing:,} missing values still present")
    
    # Check data types
    print("\nFinal data types:")
    print(df.dtypes.value_counts())
    
    # Summary statistics
    print(f"\nFinal dataset summary:")
    print(f"   • Rows: {len(df):,}")
    print(f"   • Columns: {len(df.columns)}")
    print(f"   • Unique suppliers: {df['supplier_id'].nunique() if 'supplier_id' in df.columns else 'N/A'}")
    print(f"   • Date range: {df['date'].min() if 'date' in df.columns else 'N/A'} to {df['date'].max() if 'date' in df.columns else 'N/A'}")
    
    return df

df = final_validation(df)

No missing values remaining

Final data types:
float64           11
object             7
int64              2
datetime64[ns]     1
Name: count, dtype: int64

Final dataset summary:
   • Rows: 2,994
   • Columns: 21
   • Unique suppliers: 25
   • Date range: 2015-01-01 00:00:00 to 2024-12-01 00:00:00


In [11]:
def save_cleaned_data(df, output_path='../data/processed/cleaned_data.csv'):
    """Save the cleaned dataset"""
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save as CSV
    df.to_csv(output_path, index=False)
    print(f"Saved cleaned data to {output_path}")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Also save as JSON for web app
    json_path = output_path.replace('.csv', '.json')
    df.to_json(json_path, orient='records', indent=2, date_format='iso')
    print(f"Saved cleaned data to {json_path}")
    
    return df

df = save_cleaned_data(df)

Saved cleaned data to ../data/processed/cleaned_data.csv
  Shape: 2,994 rows × 21 columns
Saved cleaned data to ../data/processed/cleaned_data.json
